<a href="https://colab.research.google.com/github/meghamkpatel/physicaltherapyassistant/blob/main/PhysioPhrame_Test_and_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import streamlit as st
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_openai.embeddings import OpenAIEmbeddings

# Load environment variables
OPENAI_API_KEY = st.secrets["OPENAI_API_KEY"]
MODEL = "gpt-3.5-turbo"

# Initialize Pinecone
PINECONE_API_KEY = st.secrets["PINECONE_API_KEY"]
pc = Pinecone(api_key=PINECONE_API_KEY)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [ ]:
from langchain_community.document_loaders import DirectoryLoader

def load_local_documents(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    
    # Replace newline characters with spaces in each document
    for doc in documents:
        doc.page_content = doc.page_content.replace("\n", " ")
    
    return documents

# Define the directory containing the documents
directory = 'content/Textbook'

# Load local documents
documents = load_local_documents(directory)

In [ ]:
from pinecone import Pinecone
from langchain_openai.embeddings import OpenAIEmbeddings

# Initialize Pinecone
pinecone_api_key = st.secrets["PINECONE_API_KEY"]
pinecone = Pinecone(api_key=pinecone_api_key)

# Create or load the index
index_name = "physical-therapy"
pc_index = pc.Index(index_name)


vectorstore = pinecone.Index(name=index_name)

# Generate embeddings
embeddings = OpenAIEmbeddings()
vector_dict = {f"doc_{i}": embeddings.embed_query(doc.page_content) for i, doc in enumerate(documents)}


In [ ]:
import pandas as pd

# Load the content in a pandas DataFrame
df = pd.DataFrame([doc.page_content for doc in documents], columns=["text"])

In [ ]:
from giskard.rag import KnowledgeBase, generate_testset, evaluate

# Create a Knowledge Base
knowledge_base = KnowledgeBase(df)

In [ ]:
# Generate the Test Set
testset = generate_testset(
    knowledge_base,
    num_questions=20,
    agent_description="A chatbot answering questions about physical therapy",
)

In [ ]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

In [ ]:
testset.save("test-set.jsonl")


In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

In [ ]:
prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models import ChatOpenAI
from operator import itemgetter

# Create the RAG Chain
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
def retriever_fn(question):
    response = pc_index.query(queries=[question],top_k=5)
    doc_ids = response[0].ids
    return [{"context": df.iloc[doc_id]["text"], "question": question} for doc_id in doc_ids]

chain = (
    {
        "context": retriever_fn,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
chain.invoke("What is Physical Therapy?")


In [ ]:
# Function to invoke the chain with a specific question
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

In [ ]:
from giskard.rag import KnowledgeBase, generate_testset, evaluate

# Evaluate the model on the test set
report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

In [ ]:
# Display the report
display(report)